## Chasing the Platypus via MG-RAST

This tutorial covers downloading all public MG-RAST datasets; parsing those files to search for a specific taxa of interest, in our case Platypus (Ornithorhynchus); and finding the geographycal location of the samples that match those samples, via Google. Note that you need [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/#Download/) installed.

## Downloading all public MG-RAST datasets

The first step is to crawl MG-RAST's website to search for all public datasets, downloading both the data (in HTML format) and the metadata (tab separated) for each study and store them locally. To avoid having hundreds of files in a single directory the script will group them in folders by the first 3 characters of its name. Note that MG-RAST calls each sample a project, here we will call them samples.

In [ ]:
from BeautifulSoup import BeautifulSoup
from urllib2 import urlopen, URLError
from os.path import isfile, exists, join
from os import makedirs
from socket import timeout as TOError

mgrast_analysis = 'http://metagenomics.anl.gov/metagenomics.cgi?page=Analysis'
mgrast_metagenome = ('http://metagenomics.anl.gov/metagenomics.cgi?page='
                     'MetagenomeOverview&metagenome=')
mgrast_metadata = ('http://metagenomics.anl.gov/metagenomics.cgi?page='
                   'MetagenomeProject&action=download_md&filetype=text&'
                   'filename=mgm%s.metadata.txt')

page = urlopen(mgrast_analysis)
soup = BeautifulSoup(page.read())
count = 0
all_inputs = soup.findAll('input', {'id': 'list_select_data_0'})[0]
print '====> Got Analysis page results'
for maingroups in all_inputs['value'].encode('utf-8').split('||'):
    for mg in maingroups.split('@'):
        sample = mg.split('##')[0].split('~#')[0]
        # will group by the first 3 digits of the project name
        dirname = sample[:3]

        sample_fp = join(dirname, 'sample_' + sample + '.txt')
        map_fp = join(dirname, 'map_' + sample + '.txt')
        
        count = count + 1
        if count % 50 == 0:
            print "====> Processed %d samples" % count
        # continue if sample is blank or file already exists
        if sample == '' or (isfile(sample_fp) and isfile(map_fp)):
            continue

        print 'Processing (%d): %s %s' % (count, sample, mg)

        if not exists(dirname):
            makedirs(dirname)

        # visiting the Overview page to download counts
        try:
            page_sample = urlopen(mgrast_metagenome + sample, timeout=40)
            page_mapping = urlopen(mgrast_metadata % sample, timeout=40)
        except (TOError, URLError), e:
            # Timeout
            print e
            count = count - 1
            continue
        
        open(sample_fp, 'w').write(page_sample.read())
        open(map_fp, 'w').write(page_mapping.read())

## Finding taxa

Now that we have all MG-RAST files in our local folder we can search for any taxa of interest. Note that the HMTL filess that we downloaded in the previous step have all data information that MG-RAST provides from a sample, for example: multilevel taxonomic and functional assignments. Thus, in the next step you can search by any "string" that fits in those categories. The output of this section is tab separated text file named as the query (lower case) with two columns: sample name and counts of the successful search on that sample.

In [ ]:
from BeautifulSoup import BeautifulSoup
from os import listdir, makedirs
from os.path import isdir, isfile, join, basename, exists

query = 'Ornithorhynchus'
query = query.lower()
output_fp = open(query + '.txt', 'w')

# looping in local dir to find downloaded folders
# all mgrast created folders are 3 chars long and start with a 4
dirs = [d for d in listdir('./')
        if len(d) == 3 and d.startswith('4') and isdir(d)]

for d in dirs:
    # getting files in current directory
    files = [join(d, f) for f in listdir(d) if f.startswith('sample_')]
    for sample in files:
        sample_id = basename(f)[len('sample_'):-len('.txt')]
        mapping = join(d, 'map_%s.txt' % sample_id)

        # validating that we have both the sample and the metadata file for
        # the given project
        if not isfile(sample) or not isfile(mapping):
            raise ValueError('Missing files %s, %s' % (sample, mapping))

        soup = BeautifulSoup(open(sample, 'U').read())
        for s in soup.findAll('script'):
            s = str(s).lower()
            
            if query in s:
                target = [t for t in s.split('\n') if query in t][0]
                counts = target.split(',')[1]
                counts = counts[:counts.find(']')]
                output_fp.write('%s\t%s\n' % (sample_id, counts))
                
output_fp.close()

## Finding geographical localtion of samples

In this step we will search for the geographical location of the successful samples from the previous step using the latitute and longitude values from the metadata.

In [ ]:
from urllib2 import urlopen
import json

from os import listdir
from os.path import isdir, isfile, join

input_fp = 'ornithorhynchus.txt'

# from http://stackoverflow.com/a/20169528/4228285
def getplace(lat, lon):
    url = ("http://maps.googleapis.com/maps/api/geocode/json?"
           "latlng=%s,%s&sensor=false" % (lat, lon))
    v = urlopen(url).read()
    j = json.loads(v)

    if len(j['results'])==0:
        return 'Undefined'

    components = j['results'][0]['address_components']
    country = town = None
    for c in components:
        if "country" in c['types']:
            country = c['long_name']

    return str(country)


countries = {}
for line in open(input_fp, 'r'):
    sample, counts = line.strip().split('\t')
    counts = int(counts)
    mapping = join(sample[:3], 'map_%s.txt' % sample)
    
    if not isfile(mapping):
        raise ValueError('Check the existance of: %s' % (mapping))
    
    lat = ''
    lon = ''
    for line in open(mapping, 'r'):
        line = line.split('\t')
        
        # sometimes the lat/lon have different comma separated values
        # will only use the first one
        if line[0] == 'Sample' and line[1] == 'longitude':
            lon = float(line[2].strip().split(',')[0].strip())
        elif line[0] == 'Sample' and line[1] == 'latitude':
            lat = float(line[2].strip().split(',')[0].strip())
        
        if not lat or not lon:
            country = 'Undefined'
        else:
            country = getplace(lat, lon)
        
        if country not in countries:
            countries[country] = counts
        else:
            countries[country] = countries[country] + counts

Once those results are parsed you could use http://jsfiddle.net/a1royfjh/ to plot the resuls by copy/pasting the result of the next block in the bottom-left section. You need to replace the lines that are all the way to the left, which contains the country and counts information.

In [ ]:
for country, counts in countries.items():
    if country == 'Undefined':
        print "// ['%s', %d]," % (country, counts)
    else:
        print "['%s', %d]," % (country, counts)